In [43]:
from ultralytics import SAM, YOLO

# Load a model
model = SAM('sam_l.pt')

# Display model information (optional)
model.info()

# Run inference with bboxes prompt
result = model("../Norway/train/images/Norway_000545.jpg",points=[1100,400])
    

Model summary: 358 layers, 312342832 parameters, 312342832 gradients

image 1/1 c:\Users\idi40\Desktop\data\RDD\AutoAnnotation\..\Norway\train\images\Norway_000545.jpg: 1024x1024 458.0ms
Speed: 6.0ms preprocess, 458.0ms inference, 1.0ms postprocess per image at shape (1, 3, 1024, 1024)


In [39]:
result[0]  

ultralytics.engine.results.Results object with attributes:

boxes: None
keypoints: None
masks: ultralytics.engine.results.Masks object
names: {0: '0'}
orig_img: array([[[244, 241, 236],
        [241, 238, 233],
        [239, 236, 231],
        ...,
        [255, 255, 255],
        [254, 254, 254],
        [  3,   3,   3]],

       [[244, 241, 236],
        [242, 239, 234],
        [240, 237, 232],
        ...,
        [255, 255, 255],
        [254, 254, 254],
        [  3,   3,   3]],

       [[245, 242, 237],
        [243, 240, 235],
        [242, 239, 234],
        ...,
        [255, 255, 255],
        [254, 254, 254],
        [  3,   3,   3]],

       ...,

       [[ 65,  66,  70],
        [ 64,  65,  69],
        [ 63,  64,  68],
        ...,
        [124, 126, 127],
        [126, 128, 129],
        [  0,   0,   0]],

       [[ 65,  66,  70],
        [ 64,  65,  69],
        [ 63,  64,  68],
        ...,
        [124, 126, 127],
        [126, 128, 129],
        [  0,   0,   0]],

 

In [44]:
import cv2
import numpy as np
import torch

def get_bounding_box(mask):
    # Convert tensor to numpy array if it's a PyTorch tensor
    if isinstance(mask, torch.Tensor):
        mask = mask.cpu().numpy()

    print(mask.shape)
    # Get the dimensions of the mask
    _ , height, width = mask.shape
    
    # Initialize extreme points of the bounding box
    min_x, min_y = width, height
    max_x, max_y = 0, 0
    
    # Iterate through each pixel in the mask
    for y in range(height):
        for x in range(width):
            if mask[0, y, x] == True:  # Assuming True indicates the object in the mask
                # Update extreme points of the bounding box
                min_x = min(min_x, x)
                min_y = min(min_y, y)
                max_x = max(max_x, x)
                max_y = max(max_y, y)
    
    # If no True pixels found, return None
    if min_x == width or min_y == height or max_x == 0 or max_y == 0:
        print("No 'True' pixels found in the mask.")
        return None
    
    # Compute width and height of the bounding box
    width_bb = max_x - min_x + 1
    height_bb = max_y - min_y + 1
    
    return min_x, min_y, width_bb, height_bb

bbox = get_bounding_box(result[0].masks.data)

(1, 1227, 3650)


In [46]:
img_path = "../Norway/train/images/Norway_000545.jpg"
import cv2
print(bbox)
def crop_image(image_path, min_x, min_y, max_x, max_y):
    # Load the image
    image = cv2.imread(image_path)
    
    # Crop the image using the bounding box coordinates
    cropped_image = image[min_y:max_y, min_x:max_x]
    
    return cropped_image

min_x, min_y, width, height = bbox
cropped_image = crop_image(img_path,min_x, min_y, min_x+width, min_y+height)

# Display the cropped image
cv2.imshow("Cropped Image", cropped_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

(0, 236, 3580, 991)


In [47]:
yolo =  YOLO("../runs/detect/train31/weights/best.pt")
result = yolo.predict(cropped_image, save=True)


0: 416x1408 6 D00s, 2 D20s, 75.8ms
Speed: 4.0ms preprocess, 75.8ms inference, 4.0ms postprocess per image at shape (1, 3, 416, 1408)
Results saved to runs\detect\predict2
